In [9]:
import pandas as pd
from scipy import sparse
import pandas as pd
import numpy as np
from numpy import var
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
from matplotlib import pyplot as plt
from scipy.stats import norm
import numpy as np
from fitter import Fitter, get_common_distributions, get_distributions
from scipy import stats
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statistics
import math

#load data
ratings = pd.read_csv('dataset/ratings.csv')
moviesPartial = pd.read_csv('dataset/movies.csv')
moviesFull = pd.read_csv('dataset/movie_dataset.csv')

#drop duplicates
moviesFull.drop_duplicates(subset = 'title', inplace=True)

#drop columns that do nothing
ratings = ratings.drop(['timestamp'], axis = 1)
moviesPartial = moviesPartial.drop(['genres'],axis=1)

#all the users are to be tested
userRatings = []
for i in range(1,611):
    userRatings.append(ratings[ratings['userId'] ==i])

#remove the year and keep the name
moviesPartial['title'] = moviesPartial['title'].map(lambda string: string[:-7])

#drop duplicates
moviesPartial.drop_duplicates(subset = 'title', inplace=True)

#helper functions:
def get_title_from_movieId(tmp):
    return moviesPartial[moviesPartial["movieId"]== tmp]["title"].to_string(index=False)

def checkTitle(tmp):
    if(tmp not in list(moviesFull["title"])):
        #The word "the" is being apended to the ends of titles
        tmp = tmp[-3:] +" "+ tmp[:-5]
        return moviesFull[moviesFull["title"] == tmp]["title"].to_string(index=False)
    else:
        return moviesFull[moviesFull["title"] == tmp]["title"].to_string(index=False)



#used to filter out movies that are not in both data sets
newUserTitles = []
newUserRatings = []
for userRating in userRatings:
    temp1 = []
    temp2 = []
    for rating, id in zip(list(userRating["rating"]), list(userRating["movieId"])):
        title = get_title_from_movieId(id)
        checked = checkTitle(title)
        if(checked != 'Series([], )'):
            temp1.append(checked)
            temp2.append(rating)
    newUserTitles.append(temp1)
    newUserRatings.append(temp2)
    


#this might lead to a referencce error
#populate infos; a list of data frames
infos = []
for user in newUserTitles:
    i = 0
    tempInfo = pd.DataFrame(columns = list(moviesFull.columns))
    for name in user:
        if(len(moviesFull[moviesFull["title"] == name]) !=0):
            tempInfo.loc[i] = moviesFull[moviesFull["title"] == name].iloc[0]
        i+=1
    infos.append(tempInfo)
    

features = ['keywords','cast','genres','director']


for info in infos:
    for feature in features:
        info[feature] = info[feature].fillna('')

colNumbers = []
for feature in features:   
    colNumbers.append(infos[0].columns.get_loc(feature))

fullDataList =[]


for frame in infos:
    fullDataList.append(frame.values.tolist())

combinedFeatures = []

for item in fullDataList:
    tmp = []
    for row in item:
        tmp.append(row[colNumbers[0]]+" "+row[colNumbers[1]]+" "+row[colNumbers[2]]+" "+row[colNumbers[3]])
    combinedFeatures.append(tmp)


def transform1(arr):
    ret = []
    for item in arr:
        if(item<=.5):
            ret.append(1/item)
        if(item>.5):
            ret.append(-1*(1/(1-item)))
    return ret

def transform2(arr):
    ret = []
    for item in arr:
        ret.append(1/item)
    return ret

X = []
y = []

for i in range(len(combinedFeatures)):
    cv = CountVectorizer() 
    count_matrix = cv.fit_transform(combinedFeatures[i]).toarray()
    cosine_sim = cosine_similarity(X = count_matrix[1:count_matrix.shape[1]]
    ,Y = [count_matrix[0]])

    ratings = newUserRatings[i][1:]
    similairities = np.reshape(cosine_sim,  (len(cosine_sim)))

    averageRating =  sum(ratings)/(len(ratings))

    tmp = []
    tmp.append(averageRating)
    tmp.append(sum(similairities)/(len(similairities)))
    tmp.append(statistics.variance(ratings))
    tmp.append(statistics.variance(similairities))
    ratingsZ = stats.zscore(ratings)
    ratingsZ = [0 if math.isnan(x) else x for x in ratingsZ]
    ratingsZ = [-item if item<0 else item for item in ratingsZ]
    simsZ  = stats.zscore(similairities)
    simsZ = [-item if item<0 else item for item in simsZ]

    totalRating = 0

    #does adding constant multiplier help?
    #nots: 20
    for sim, rating,simZ, ratingZ in zip(similairities, ratings, simsZ, ratingsZ):
        totalRating += sim*(rating-averageRating)*simZ*ratingZ
    
    tmp.append(totalRating)
    #next try just total rating
    X.append(tmp)
    y.append(newUserRatings[i][0])



#new varriables to feed into the linear reg model:
#variance of ratings, variance of similairities
#mean of ratings, mean of similarities, rating diff

reg = LinearRegression().fit(X, y)
#note: uses training data to test
print("after linear regression:", reg.score(X, y))
predicted = reg.predict(X)
for item, actual in zip(predicted, y):
    print("predicted:", item)
    print("actual:", actual)









after linear regression: 0.240741756905692
predicted: 4.398479212337375
actual: 4.0
predicted: 4.2053387535999
actual: 3.0
predicted: 2.041309705324788
actual: 0.5
predicted: 3.4416524342298227
actual: 3.0
predicted: 3.762484014010252
actual: 4.0
predicted: 3.8397484095111296
actual: 3.0
predicted: 3.5077421104632003
actual: 4.5
predicted: 3.913402805411114
actual: 2.0
predicted: 3.5346492791199617
actual: 3.0
predicted: 3.297025079825395
actual: 1.0
predicted: 3.9662974723075357
actual: 3.0
predicted: 4.608920949488336
actual: 4.0
predicted: 3.8017719846489264
actual: 3.0
predicted: 3.3438766573745946
actual: 1.0
predicted: 3.3354129500854266
actual: 2.5
predicted: 3.834692869308765
actual: 4.0
predicted: 4.243162001706885
actual: 4.5
predicted: 3.615784546292203
actual: 3.5
predicted: 3.12779518977471
actual: 4.0
predicted: 3.7910551583887764
actual: 4.0
predicted: 3.6005459883435322
actual: 3.5
predicted: 2.6673592306195686
actual: 5.0
predicted: 3.697563628298141
actual: 4.0
predic